# Лабораторная работа №1

$\left\{ \omega_i \right\}$ --- равномерно распределённые на $ \left[ 0, 1 \right] $.

$N \left( 0, 1 \right) $ можно генерировать из последовательности $\left\{ \omega_i \right\} $:
$ \xi_1 = \sqrt{-2 \ln \omega_1} \cdot \sin \left( 2 \pi \cdot \omega_2 \right) ,
\xi_2 = \sqrt{-2 \ln \omega_1} \cdot \cos \left( 2 \pi \cdot \omega_2 \right) $ --- независимы.

Пусть наблюдается выборка $ \vec{X} = \left( X_1, \dotsc, X_n \right)$, где $X_i \sim N \left( 0, 1 \right) $.

Расчёты будем вести при $\gamma = 0.05$.
Это означает, что уровень доверия --- $0.95, z_{jamma} = 1.96$ (для нормального распределения).

$n = 100, 1000, 10000, 100000$.

In [1]:
from numpy.random import normal, uniform
from numpy import mean, std, log, exp, array
from scipy.stats import norm, t, chi2
from math import sqrt
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [2]:
a = 0
sigma = 1
gamma = 0.05

## Задание 1
Определить скорость сходимости.
Действительно ли она $\frac{1}{\sqrt{n}}$?
Всегда ли доверительный интервал накрывает $0$?
Строим доверительный интервал в следующих случаях:

### а)
для математического ожидания $a$, когда дисперсия известна ($=1$).

Рассмотрим случай нормально распределённой генеральной совокупности, то есть допустим,
что $ \xi \sim N \left( a, \sigma^2 \right)$.

**Лемма 1** Введём обозначения для среднего арифметического
$$\hat{a}_n = \frac{1}{n} \sum \limits_{i=1}^n X_i$$
и для выборочной дисперсии
$$\hat{\sigma}_n^2 = \frac{1}{n} \sum \limits_{i=1}^n \left( X_i - \hat{a}_n \right)^2.$$
Тогда
$$\hat{a}_n \sim N \left( a, \frac{\sigma^2}{n}\right).$$
Если случайные величины нормально распределены, то их сумма нормально распределена.

Введём
$$\hat{U}_n = \frac{\hat{\sigma}_n^2}{\sigma^2} \cdot n.$$
Эта случайная величина независима с $\hat{a}_n, \hat{U}_n \sim \chi^2$ с $n-1$ степенями свободы.

Плотность $\chi^2$ распределения с $n$ степенями свободы:
$$
t_n \left( u \right) =
\frac{1}{2^{\frac{n}{2}} \cdot \Gamma \left( \frac{n}{2}\right)} \cdot u^{\frac{n}{2} - 1} \cdot e^{-\frac{u}{2}},
u \geq 0,
\Gamma \left( \alpha \right) = \int \limits_0^{\infty } u^{\alpha - 1} \cdot e^{-u} du.
$$

Используем первый пункт леммы 1, или
$$\hat{V}_n = \frac{\hat{a}_n - a}{\sigma} \cdot \sqrt{n} \sim N \left( 0, 1 \right).$$
Это означает, что $P \left\{ \left| \hat{V}_n \right| < z\right\} = 2 \Phi \left( z \right) = 1 - \gamma,$
где
$$\Phi \left( z \right) = \frac{1}{\sqrt{2\pi}} \int \limits_0^z e^{-\frac{u^2}{2}} du, z > 0.$$
Это функция Лапласа.

Из уравнения $2\Phi \left( z \right) = 1 - \gamma$ по таблицам находим $z_{\gamma}$.
Тогда $P \left\{ \left| \hat{V}_n \right| < z_{\gamma }\right\} = 1 - \gamma$.

Решаем уравнение относительно $a$.
Получим симметрический доверительный интервал
$$\hat{a}_n - \frac{z_{\gamma} \cdot \sigma}{\sqrt{n}} <
a <
\hat{a}_n + \frac{z_{\gamma} \cdot \sigma}{\sqrt{n}}.$$

Можем не попасть в этот интервал с вероятностью $\gamma$.

Абсолютная погрешность:
$$\frac{z_{\gamma} \cdot \sigma}{\sqrt{n}}.$$

In [3]:
def mean_interval_known_sigma(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    z = norm.ppf(1 - gamma / 2.)
    left = a_n - z * sigma / sqrt(n)
    right = a_n + z * sigma / sqrt(n)
    length = right - left
    print "mean 95%% confidence interval (theory), n =", n, (left, right)
    print "mean 95%% confidence interval (python), n =", n, _zconfint_generic(a_n, sigma / sqrt(n), gamma, 'two-sided')
    print "interval length:", length
    return length

In [4]:
l1 = mean_interval_known_sigma(100)

mean 95%% confidence interval (theory), n = 100 (-0.22586621174713772, 0.16612658516087309)
mean 95%% confidence interval (python), n = 100 (-0.22586621174713772, 0.16612658516087309)
interval length: 0.391992796908


In [5]:
l2 = mean_interval_known_sigma(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

mean 95%% confidence interval (theory), n = 1000 (-0.10055534179565426, 0.023403664665258062)
mean 95%% confidence interval (python), n = 1000 (-0.10055534179565426, 0.023403664665258055)
interval length: 0.123959006461
length ratio: 0.316227766017
1 / sqrt(n2 / n1): 0.316227766017


In [6]:
l3 = mean_interval_known_sigma(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n2):", 1 / sqrt(10)

mean 95%% confidence interval (theory), n = 10000 (0.0053204808787135557, 0.044519760569514635)
mean 95%% confidence interval (python), n = 10000 (0.0053204808787135557, 0.044519760569514635)
interval length: 0.0391992796908
length ratio: 0.316227766017
1 / sqrt(n3 / n2): 0.316227766017


In [7]:
l4 = mean_interval_known_sigma(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

mean 95%% confidence interval (theory), n = 100000 (-0.0073890186655487973, 0.0050068819805424332)
mean 95%% confidence interval (python), n = 100000 (-0.0073890186655487973, 0.0050068819805424332)
interval length: 0.0123959006461
length ratio: 0.316227766017
1 / sqrt(n4 / n3): 0.316227766017


При всех $n$ доверительный интервал накрывает $0$.
Скорость сходимости доверительного интервала к истинному значению математического ожидания $a = 0$ равна
$1 / \sqrt{n}$.
Результаты,
полученные с помощью теоретических выкладок и функции `_zconfint_generic(mean, std_mean, gamma, alternative)`
совпадают.

### б)
для математического ожидания $a$, когда дисперсия неизвестна.

**Лемма 2** Определим
\begin{equation}
\hat{V}_n = \frac{\hat{a}_n - a}{ \sigma} \cdot \sqrt{n}.
\end{equation}
Тогда
\begin{equation}
\hat{Z}_n = \sqrt{n - 1} \cdot \frac{\hat{V}_n}{\sqrt{\hat{U}_n}}
\end{equation}
имеет распределение Стьюдента с $n-1$ степенью свободы.
Плотность распределения:
\begin{equation}
S_n \left( u \right) =
\frac{\Gamma \left( \frac{n + 1}{2}\right)}{\sqrt{n \cdot \pi} \cdot \Gamma \left( \frac{n}{2}\right)} \cdot
\left( 1 + \frac{u^2}{n} \right)^{-\frac{n+1}{2}},
-\infty < u < +\infty.
\end{equation}

Используем лемму 2
\begin{equation}
\hat{Z}_n =
\sqrt{n-1} \cdot \frac{\sqrt{n} \cdot
\left( \hat{a}_n - a\right) \cdot \sigma}{\sigma \cdot \sqrt{n} \cdot \hat{\sigma}_n} =
\sqrt{n - 1} \cdot \frac{\hat{a}_n - a}{\hat{\sigma}_n}.
\end{equation}
Получается, что
\begin{equation}
P \left\{ \left| \hat{Z}_n\right| < z \right\} = 2 \int \limits_0^1 S_{n-1} \left( u \right) du = 1 - \gamma.
\end{equation}
Решаем:
\begin{equation}
\left| \hat{Z}_n\right| < z_{\gamma} \Rightarrow
\hat{a}_n - \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n-1}} <
a <
\hat{a}_n + \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n-1}}.
\end{equation}
Отличия:
- дисперсию $\sigma$ заменяем на оценку $\hat{\sigma}_n$;
- $\sqrt{n} \to \sqrt{n-1}$;
- $N \to S_{n-1}$, то есть $z_{gamma}$ вычисляется по другим таблицам.

In [8]:
def mean_interval_unknown_sigma(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    sigma_n = sqrt(((X - a_n) ** 2).mean())
    z = t.ppf(1 - gamma / 2., n - 1)
    left = a_n - z * sigma_n / sqrt(n - 1)
    right = a_n + z * sigma_n / sqrt(n - 1)
    length = right - left
    print "mean 95%% confidence interval (theory), n =", n, (left, right)
    print "mean 95%% confidence interval (python), n =", n,  _tconfint_generic(a_n, sigma_n, n - 1, gamma, 'two-sided')
    print "interval length:", length
    return length

In [9]:
l1 = mean_interval_unknown_sigma(100)

mean 95%% confidence interval (theory), n = 100 (-0.19722043090979052, 0.17294090202423873)
mean 95%% confidence interval (python), n = 100 (-1.8536691443027762, 1.8293896154172244)
interval length: 0.370161332934


In [10]:
l2 = mean_interval_unknown_sigma(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

mean 95%% confidence interval (theory), n = 1000 (-0.068099668682589348, 0.052758792220641296)
mean 95%% confidence interval (python), n = 1000 (-1.9176547839996658, 1.9023139075377178)
interval length: 0.120858460903
length ratio: 0.326502122589
1 / sqrt(n2 / n1): 0.316227766017


In [11]:
l3 = mean_interval_unknown_sigma(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n2):", 1 / sqrt(10)

mean 95%% confidence interval (theory), n = 10000 (-0.017118923895262688, 0.021841870634974159)
mean 95%% confidence interval (python), n = 10000 (-1.9455808487204898, 1.9503037954602014)
interval length: 0.0389607945302
length ratio: 0.322367124644
1 / sqrt(n3 / n2): 0.316227766017


In [12]:
l4 = mean_interval_unknown_sigma(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

mean 95%% confidence interval (theory), n = 100000 (-0.0074450725428442284, 0.0049623994501167765)
mean 95%% confidence interval (python), n = 100000 (-1.9630251026792367, 1.9605424295865093)
interval length: 0.012407471993
length ratio: 0.318460445752
1 / sqrt(n4 / n3): 0.316227766017


Скорость сходимости доверительного интервала к истинному значению математического ожидания $a = 0$ равна
$\frac{1}{\sqrt{n}}$.
Доверительный интервал всегда накрывает $0$.
В данном случае теоретические результаты отличаются от результатов,
полученных с помощью функции `_tconfint_generic(mean, std_mean, dof, gamma, alternative)` отличаются,
так как в функции нет деления на $\sqrt{n-1}$.

### в)
для математического ожидания $a$, когда распределение не $N \left( 0, 1 \right)$, то есть мы не знаем,
что распределение нормальное.
Просто есть последовательность $\vec{X}$.

Считаем, что $M \xi = a, D \xi < \infty $.

Пользуемся центральной предельной теоремой.

Пусть несмещённая оценка дисперсии (выборочная несмещённая дисперсия)
\begin{equation}
\hat{\sigma}_n = \frac{1}{n - 1} \cdot \left( \sum \limits_{i = 1}^n X_i^2 - \frac{1}{n} \cdot \hat{a}_n^2 \right).
\end{equation}
Согласно центральной предельной теореме выходит
\begin{equation}
P \left\{ \sqrt{n} \cdot \frac{\left| \hat{a}_n - a \right|}{\sigma} < z \right\}
\overset{n \to \infty }{\to} \Phi \left( z \right) =
1 - \gamma.
\end{equation}
В этом случае будем строить ассимптотические доверительные интервалы
\begin{equation}
\hat{a}_n - \frac{z_{\gamma} \cdot \sigma}{\sqrt{n}} < a < \hat{a}_n + \frac{z_{\gamma} \cdot \sigma}{ \sqrt{n}}.
\end{equation}
Проблема в том, что $\sigma$ неизвестно.
На практике заменяют $\sigma$ на $\hat{\sigma}_n$, то есть доверительный интервал строится таким образом
\begin{equation}
\hat{a}_n = \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n}} <
a <
\hat{a}_n + \frac{z_{\gamma} \cdot \hat{\sigma}_n}{\sqrt{n}},
\end{equation}
где $z_{\gamma}$ находится через функцию Лапласа.

In [13]:
def mean_interval_unknown_distribution(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    z = norm.ppf(1 - gamma)
    left = a_n - z * sigma / sqrt(n)
    right = a_n + z * sigma / sqrt(n)
    length = right - left
    print "mean 95%% confidence interval, n =", n, (left, right)
    print "interval length:", length
    return length

In [14]:
l1 = mean_interval_unknown_distribution(100)

mean 95%% confidence interval, n = 100 (0.027462901682638069, 0.35643362707293247)
interval length: 0.32897072539


In [15]:
l2 = mean_interval_unknown_distribution(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

mean 95%% confidence interval, n = 1000 (-0.039023972257217397, 0.065005705317894069)
interval length: 0.104029677575
length ratio: 0.316227766017
1 / sqrt(n2 / n1): 0.316227766017


In [16]:
l3 = mean_interval_unknown_distribution(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n2):", 1 / sqrt(10)

mean 95%% confidence interval, n = 10000 (-0.0074752765335276208, 0.025421796005501825)
interval length: 0.032897072539
length ratio: 0.316227766017
1 / sqrt(n3 / n2): 0.316227766017


In [17]:
l4 = mean_interval_unknown_distribution(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

mean 95%% confidence interval, n = 100000 (-0.003090645935919172, 0.0073123218215919751)
interval length: 0.0104029677575
length ratio: 0.316227766017
1 / sqrt(n4 / n3): 0.316227766017


Скорость сходимости доверительного интервала к истинному значению математического ожидания $a = 0$ равна
$\frac{1}{\sqrt{n}}$.
Доверительный интервал всегда накрывает $0$.

## Задание 2
При заданных $n$ строим доверительный интервал для дисперсии $\sigma^2 = 1$ (в допущении $N \left( 0, 1 \right)$).
Действительно ли он сужается и с какой скоростью?
Действительно ли этот интервал накрывает единицу?

Используем вторую часть первой леммы.

В данном случае дисперсия принимает значения только больше нуля, $\chi^2$ распределение не симметрично.
Оно имеет вид:
![chi-squared distribution](chi-squared.jpg)
Нужно строить такой доверительный интервал,
чтобы расстояние между точками $z_{\gamma}^{\left( 1 \right)}$ и $z_{\gamma}^{\left( 2 \right)}$ было маленьким:
\begin{equation}
\int \limits_0^{z_{\gamma}^{\left( 1 \right)}} t_{n-1} \left( u \right) du = \frac{\gamma}{2}, 
\int \limits_{z_{\gamma}^{\left( 2 \right)}} t_{n - 1} \left( u \right) du = \frac{\gamma}{2}.
\end{equation}
При заданном $\gamma$ из этих уравнений находим $z_{\gamma}^{\left( 1 \right) }$ и $z_{\gamma}^{\left( 2 \right)}$.
Получается,
что $P \left\{ z_{\gamma}^{\left( 1 \right) } < \hat{U}_n < z_{\gamma}^{\left( 2 \right) } \right\} = 1 - \gamma$.

Доверительный интервал для дисперсии
\begin{equation}
\frac{\left( n - 1 \right) \cdot \hat{\sigma}_n^2}{z_{\gamma n}^{\left( 2 \right)}} <
\sigma^2 <
\frac{\left( n - 1 \right) \cdot \hat{\sigma}_n^2}{z_{\gamma n}^{\left( 2 \right) }}.
\end{equation}
Когда увеличиваем $n$, увеличивабтся $z_{\gamma n}^{\left( i \right) }$.
Сам интервал сужается.

In [18]:
def variance_interval(n):
    X = normal(a, sigma, n)
    a_n = X.mean()
    sigma_n = (sum(X ** 2) - mean(a_n ** 2)) / (n - 1)
    z1 = chi2.ppf(gamma / 2, n - 1)
    z2 = chi2.ppf(1 - gamma / 2, n - 1)
    left = (n - 1) * sigma_n / z2
    right = (n - 1) * sigma_n / z1
    length = right - left
    print "variance 95%% confidence interval, n =", n, (left, right)
    print "interval length:", length
    return length

In [19]:
l1 = variance_interval(100)

variance 95%% confidence interval, n = 100 (0.84174142333469648, 1.4735075768606514)
interval length: 0.631766153526


In [20]:
l2 = variance_interval(1000)
print "length ratio:", l2 / l1
print "1 / sqrt(n2 / n1):", 1 / sqrt(10)

variance 95%% confidence interval, n = 1000 (0.89728198338203757, 1.0693953437372099)
interval length: 0.172113360355
length ratio: 0.272432069041
1 / sqrt(n2 / n1): 0.316227766017


In [21]:
l3 = variance_interval(10000)
print "length ratio:", l3 / l2
print "1 / sqrt(n3 / n1):", 1 / sqrt(10)

variance 95%% confidence interval, n = 10000 (0.96206567959350731, 1.0169104822742252)
interval length: 0.0548448026807
length ratio: 0.318655115254
1 / sqrt(n3 / n1): 0.316227766017


In [22]:
l4 = variance_interval(100000)
print "length ratio:", l4 / l3
print "1 / sqrt(n4 / n3):", 1 / sqrt(10)

variance 95%% confidence interval, n = 100000 (0.99045078659122077, 1.0079670948073913)
interval length: 0.0175163082162
length ratio: 0.319379546648
1 / sqrt(n4 / n3): 0.316227766017


Интервал сужается к истинному значению дисперсии $\sigma^2 = 1$ со скоростью $\frac{1}{\sqrt{n}}$.
Этот интервал действительно покрывает единицу.

## Задание 3
Хотим вычислить величину
$I = P \left( \xi < \eta \right), \xi \sim F \left( u \right), \eta \sim G \left( u \right)$,
где $F \left( u \right)$ и $G \left( u \right) $ --- функции распределения.
$\xi \geq 0$ и $\eta > 0$ --- независимы.
Фиксируем $\xi$:
\begin{equation}
I = \int \limits_0^{\infty} F \left( u \right) dG \left( u \right) = MF \left( \eta \right).
\end{equation}
Фиксируем $\xi$:
\begin{equation}
I = \int \limits_0^{\infty } \left[ 1 - G \left( u \right) \right] dF \left( u \right) =
M \left[ 1 - G \left( \xi \right)\right].
\end{equation}
Если $\xi \gg \eta$ в среднем, то $\xi < \eta$ очень редко.

В первом случае дисперсия будет пропорциональна $p^2$.

Возьмём конкретные функции распределения: $F \left( u \right) = 1 - e^{- \left( \alpha \cdot u\right)^2}, u \geq 0$ --- распределение Вейбулла (Weibull), $G \left( u \right) = 1 - e^{-u^3}, u \geq 0$.

Чтобы сгенерировать $\xi$, нужно приравнять $1 - e^{-\left( \alpha \cdot u \right)^2}$ к $1 - \omega $,
чтобы найти $u$, то есть
\begin{equation}
\xi = -\frac{1}{\alpha} \cdot \left( \ln \omega \right)^{\frac{1}{2}} \sim F \left( u \right).
\end{equation}

$P \left\{ \xi < u\right\} =
P \left\{ F^{-1} \left( \omega \right) < u \right\} =
P \left\{ \omega < F \left( u \right) \right\} =
F \left( u \right),
\eta = - \left( \ln \omega \right)^{\frac{1}{3}}$.

Хотим вычислить интеграл методом Монте-Карло.
Уровень доверия возьмём $\gamma = 0.05$.

Относительную погрешность возьмём 1%, то есть $\varepsilon = 0.01$.

Для первого способа оценка интеграла
\begin{equation}
\hat{I}_n^{\left( 1 \right) } = \frac{1}{n} \sum \limits_{i=1}^n F \left( \eta_i \right),
\alpha = 0.1, 0.01, 0.001. 
\end{equation}
Вычислить $I$ точно.
Для каждого из способов найти $n$, когда относительная погрешность оценки --- 1%
\begin{equation}
n^* = \min n \geq n_0: n \geq \frac{z_{\gamma}^2 \cdot \hat{\sigma}_n^2}{\varepsilon^2 \cdot \hat{I}_n^2}.
\end{equation}
В первом случае $n_0 = 100$.

Во втором случае $n_0$ в зависимости от $\alpha$ нужно увеличивать, например, $n_0 = 1000, 10000, 100000$,
чтобы дисперсия была устойчивой.

In [23]:
epsilon = 0.01

In [24]:
def generation(n, alpha):
    omega = uniform(size=n)
    xi = [1 / alpha * sqrt(abs(log(x))) for x in omega]
    G = 1 - array([1 - exp(-x ** 3) for x in xi])
    eta = [-(log(x)) ** (1 / 3) for x in omega]
    F = array([1 - exp(-(alpha * x) ** 2) for x in eta])
    return F, G

In [25]:
n = 100
alpha = 0.1
F, G = generation(n, alpha)

In [26]:
def integrals(n, alpha):
    F, G = generation(n, alpha)
    return mean(F), mean(G)

In [27]:
print "(I1, I2):", integrals(n, alpha)

(I1, I2): (0.0099501662508318933, 0.019029948596360778)


In [28]:
def unknown_distribution(sample, n, alpha, i):
    a_n = sample.mean()
    sigma_n = (sum(sample ** 2) - mean(a_n ** 2)) / (n - 1)
    z = norm.ppf(1 - gamma)
    left = a_n - z * sigma / sqrt(n)
    right = a_n + z * sigma / sqrt(n)
    print "alpha:", alpha
    print "n:", n
    print "mean 95%% confidence interval for integral:", (left, right)
    print "n >=", (z * sigma_n / (epsilon * integrals(n, alpha)[i])) ** 2 // 1 + 1

In [29]:
unknown_distribution(generation(n, alpha)[0], n, alpha, 0)

alpha: 0.1
n: 100
mean 95%% confidence interval for integral: (-0.15453519644431532, 0.17443552894597911)
n >= 3.0


In [30]:
n = 100
alpha = 0.1
unknown_distribution(generation(n, alpha)[1], n, alpha, 1)

alpha: 0.1
n: 100
mean 95%% confidence interval for integral: (-0.15323347150721828, 0.17573725388307615)
n >= 7719786402.0


Значения интегралов попадают в построенные доверительные интервалы.
Минимальное значение $n$ в первом случае мало, так как все элементы выборки $\eta$ оказались одинаковыми.